In [1]:
import ocr_module
import ocr_postprocess
import medication_matching

[2025/02/21 13:41:19] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Model... 




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
image_text = ocr_module.extract_image_text('test2.jpg')

[2025/02/21 13:41:54] ppocr DEBUG: dt_boxes num : 10, elapsed : 1.1224629878997803
[2025/02/21 13:41:55] ppocr DEBUG: cls num  : 10, elapsed : 0.2727363109588623
[2025/02/21 13:41:57] ppocr DEBUG: rec_res num  : 10, elapsed : 2.3669161796569824


In [7]:
image_text

'Levothyroxine sodium Euthyrox 50 mcg Tablet Thyroid Hormone Replacement CK R Imported by Merck Inc. 36th Floor,The Finance Center 26th Street corner 9th Avenue, Bonifacio Global City,Taguig 100 Tablets MERCK'

In [4]:
extracted_text = ocr_postprocess.extract_info_from_ocr(image_text)

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Extract the medication name from this OCR text of a medicine package:
    Levothyroxine sodium Euthyrox 50 mcg Tablet Thyroid Hormone Replacement CK R Imported by Merck Inc. 36th Floor,The Finance Center 26th Street corner 9th Avenue, Bonifacio Global City,Taguig 100 Tablets MERCK 

    Return only the generic medication name, manufacturer/laboratory, dosage, and packaging quantity of the OCR text, don't add addresses or any labeling. Return them seperated by |.
system
Levothyroxine sodium|Merck Inc.|50 mcg|36 tablets


In [6]:
medication_matching.get_info(extracted_text)

[{'INDEX': 5892,
  'Registration Number': 'DRP-7270',
  'Generic Name': 'Levothyroxine (as sodium)',
  'Brand Name': 'Zaliana',
  'Dosage Strength': '50 mg',
  'Dosage Form': 'Tablet',
  'Classification': 'Prescription Drug (RX)',
  'Packaging': "Amber Glass Bottle x 100 tablets (Box of 1's)",
  'Pharmacologic Category': 'Thyroid Hormone Replacement',
  'Manufacturer': 'Sterile-Gene Life Sciences (P) Ltd.',
  'Country of Origin': 'India',
  'Trader': nan,
  'Importer': 'Lloyd Laboratories, Inc.',
  'Distributor': 'Innogen Pharmaceuticals, Inc.',
  'Application Type': 'Renewal',
  'Issuance Date': '12-Mar-24',
  'Expiry Date': '30-Jul-29',
  'match_score': 78,
  'rx_info': {'INDEX': 4527,
   'Drug_Name': 'Levothyroxine Sodium (Levoxyl)',
   'URL': 'https://www.rxlist.com/levoxyl-drug.htm'}},
 {'INDEX': 14825,
  'Registration Number': 'DRP-14562',
  'Generic Name': 'Levothyroxine (as sodium)',
  'Brand Name': nan,
  'Dosage Strength': '100 mcg',
  'Dosage Form': 'Tablet',
  'Classificati

In [7]:
import medication_matching

## LLM Testing

In [1]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from transformers import TextStreamer
import os
import torch

model_id = 'llm'

tokenizer = AutoTokenizer.from_pretrained(model_id)
print("Loading Model... \n\n")
model = AutoModelForCausalLM.from_pretrained(
    model_id
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Model... 




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
def extract_info_from_ocr(ocr_text):

    inst = f"""Extract the medication name from this OCR text of a medicine package:
    {ocr_text} 

    Return only the generic medication name, manufacturer/laboratory, dosage, and packaging quantity of the OCR text (e.g. 40 tablets, blister pack, etc.). Return them seperated by |."""

    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": inst},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_text=tokenizer.apply_chat_template(messages, tokenize=False)

    # Tokenize the sample
    inputs = tokenizer([input_text], return_tensors='pt')

    # Call generate on the inputs
    out = model.generate(
        **inputs,
        max_new_tokens=96,
        streamer=TextStreamer(tokenizer=tokenizer, skip_special_tokens=True),
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False
    )

    extracted_query = tokenizer.batch_decode(out)[0]
    #extracted_query = extracted_query[extracted_query.index('<|im_end|>\n<|im_start|>system\n')+len('<|im_end|>\n<|im_start|>system\n'):]
    #extracted_query = extracted_query.replace('<|im_end|>', '')
    extracted_query = extracted_query[len(input_text):]
    extracted_query = extracted_query.replace("<|im_start|>system", "").replace("<|im_end|>", "")
    return extracted_query.strip()


In [22]:
result = "Levothyroxine sodium Euthyrox 50 mcg Tablet Thyroid Hormone Replacement CK R Imported by Merck Inc. 36th Floor,The Finance Center 26th Street corner 9th Avenue, Bonifacio Global City,Taguig 100 Tablets MERCK"
extracted_result = extract_info_from_ocr(result)
extracted_result

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Extract the medication name from this OCR text of a medicine package:
    Levothyroxine sodium Euthyrox 50 mcg Tablet Thyroid Hormone Replacement CK R Imported by Merck Inc. 36th Floor,The Finance Center 26th Street corner 9th Avenue, Bonifacio Global City,Taguig 100 Tablets MERCK 

    Return only the generic medication name, manufacturer/laboratory, dosage, and packaging quantity of the OCR text (e.g. 40 tablets, blister pack, etc.). Return them seperated by |.


c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\Lib\site-packages\transformers\generation\configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should s

system
Levothyroxine sodium|Merck Inc.|50 mcg|36th Floor, The Finance Center


'Levothyroxine sodium|Merck Inc.|50 mcg|36th Floor, The Finance Center'

In [18]:
extracted_result

'<|im_start|>user\nLevothyroxine sodium|Merck Inc.|50 mcg|36th Floor, The Finance Center'